In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rajna\\OneDrive\\Desktop\\DLP001\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rajna\\OneDrive\\Desktop\\DLP001'

In [5]:
# pip install dagshub

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/rishabh6900/DLP001.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="rishabh6900"
os.environ["MLFLOW_TRACKING_PASSWORD"]="4bcf9bdf86a4863b89822ffd8ed5b9ddcd8b46f2 "

In [47]:
import dagshub
dagshub.init(repo_owner='rishabh6900', repo_name='DLP001', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\rajna\anaconda3\envs\BT\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=cfdf5ed5-4d35-4299-8924-8e64db58a367&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=eacb487475586358dbb94afda25fc58a109670b473ab79cd9619060da7985ab8


[2025-02-23 10:23:44,372: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2025-02-23 10:23:45,441: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2025-02-23 10:23:47,857: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as rishabh6900

[2025-02-23 10:23:47,875: INFO: helpers: Accessing as rishabh6900]
[2025-02-23 10:23:48,885: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/rishabh6900/DLP001 "HTTP/1.1 200 OK"]
[2025-02-23 10:23:49,857: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "rishabh6900/DLP001"

[2025-02-23 10:23:49,865: INFO: helpers: Initialized MLflow to track repo "rishabh6900/DLP001"]


Repository rishabh6900/DLP001 initialized!

[2025-02-23 10:23:49,870: INFO: helpers: Repository rishabh6900/DLP001 initialized!]


In [48]:
import tensorflow as tf

In [49]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [50]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model = Path 
    training_data = Path 
    all_params: dict
    mlflow_uri: str 
    params_image_size: list 
    params_batch_size: int

In [51]:
from src.cnnClassifier.config.configuration import ConfigurationManager

In [52]:
from src.cnnClassifier.components import * 
from src.cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [54]:
CONFIG_FILE_PATH = "config/config.yaml"
PARAMS_FILE_PATH = "params.yaml"


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Brain-Tumor",
            mlflow_uri="https://dagshub.com/rishabh6900/DLP001.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [55]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [50]:
### 2:30 

In [56]:
config_filepath = "config/config.yaml"

In [58]:
class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config = config
    

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',  # Since you have multiple categories
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [59]:
from pathlib import Path

try:
    config = ConfigurationManager(
        config_filepath=Path(config_filepath),
        params_filepath=Path(PARAMS_FILE_PATH)
    )
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-02-23 10:24:34,226: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-23 10:24:34,229: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-23 10:24:34,230: INFO: common: created directory at: artifacts]


TypeError: __init__() got an unexpected keyword argument 'path_of_model'